# Peer-graded Assignment: Эксперименты с моделью

На прошлой неделе вы поучаствовали в соревновании на kaggle и, наверняка, большинство успешно справилось с прохождением baseline, а значит пора двигаться дальше - заняться оптимизацией модели, провести серию экспериментов и построить сильное финальное решения.

В этом задании вам нужно провести ряд эскпериментов, оценить качество полученных в процессе экспериментирования моделей и выбрать лучшее решение. 

Задание будет оцениваться на основании загруженного jupyther notebook и развернутых ответов на поставленные вопросы.

# Данные

In [ ]:
# !unzip data.zip

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import os
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector as selector, ColumnTransformer

# PATH_TO_DATA = 'data'
PATH_TO_DATA = '../input/churn-prediction-spec'

In [ ]:
df = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_data.train'))
y = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_labels.train'), header=None, squeeze=True)

In [ ]:
# cat_cols = df.select_dtypes(include='object').columns
# df[cat_cols] = df[cat_cols].astype('category')

In [ ]:
df_train, df_ho, y_train, y_valid = train_test_split(df, y, test_size=5000)

In [ ]:
class NanColumnsDropper(BaseEstimator, TransformerMixin):
    def __init(self):
        super().__init__()

    def fit(self, X):
        self.nan_cols = X.loc[:, X.isna().all()].columns
        return self

    def transform(self, X):
        X_transformed = X.drop(self.nan_cols, axis=1)
        self.cols = X_transformed.columns
        return X_transformed

In [ ]:
numeric_transformer = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='most_frequent')),
    # ('ordinal_encoder', OrdinalEncoder())
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, selector(dtype_exclude="object")),
    ('cat', categorical_transformer, selector(dtype_include="object"))
])

preprocessor_pipeline = Pipeline([
    ('nan_columns_dropper', NanColumnsDropper()),
    ('preprocessor', preprocessor),
    # ('scaler', StandardScaler())
])

In [ ]:
%%time
X_train_all = preprocessor_pipeline.fit_transform(df)
X_train_all = pd.DataFrame(X_train_all, columns=preprocessor_pipeline['nan_columns_dropper'].cols)#.astype('int')
X_train = preprocessor_pipeline.fit_transform(df_train)
X_train = pd.DataFrame(X_train, columns=preprocessor_pipeline['nan_columns_dropper'].cols)#.astype('int')
X_valid = preprocessor_pipeline.fit_transform(df_ho)
X_valid = pd.DataFrame(X_valid, columns=preprocessor_pipeline['nan_columns_dropper'].cols)#.astype('int')
X_train

# Модель

In [ ]:
# !pip install catboost

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit
from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, roc_auc_score

from catboost import CatBoostClassifier

In [ ]:
cv = StratifiedShuffleSplit(n_splits=8, random_state=2179)

In [ ]:
catboost_classifier = CatBoostClassifier(
    max_depth=6, 
    n_estimators=100, 
    cat_features=df_train.select_dtypes(include='object').columns.to_list()
)

In [ ]:
%%time
catboost_classifier.fit(X_train, y_train)
print(roc_auc_score(y_valid, catboost_classifier.predict_proba(X_valid)[:, 1]))

In [ ]:
catboost_classifier = CatBoostClassifier(
    max_depth=6, 
    n_estimators=100, 
    cat_features=df_train.select_dtypes(include='object').columns.to_list(),
    logging_level='Silent'
)

In [ ]:
%%time
scores = cross_val_score(estimator=catboost_classifier, X=X_train, y=y_train, cv=cv, n_jobs=-1, 
                         scoring='roc_auc')
print(scores, '\n', scores.mean())

In [ ]:
%%time
gb_classifier = GradientBoostingClassifier(random_state=2179)
gb_classifier.fit(X_train, y_train)
print(roc_auc_score(y_valid, gb_classifier.predict_proba(X_valid)[:, 1]))

## Инструкции

1\. Начнем с простого. Давайте оценим как много объектов действительно нужно для построения качественной модели. Для обучения доступна достаточно большая выборка и может так оказаться, что начиная с некоторого момента рост размера обучающей выборки перестает влиять на качество модели. Постройте кривые обучения, обучая модель на выборках разного размера начиная с небольшого количество объектов в обучающей выборке и постепенно наращивая её размер с некоторым шагом. Обратите внимание на `sklearn.model_selection.learning_curve`

In [ ]:
from sklearn.model_selection import  learning_curve

from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (13, 8)
import seaborn as sns
sns.set()

In [ ]:
%%time
train_sizes_abs, train_scores, test_scores, fit_times, score_times = learning_curve(
    estimator=catboost_classifier,
    X=X_train,
    y=y_train,
    cv=cv,
    scoring='roc_auc',
    n_jobs=-1,
    return_times=True
)

In [ ]:
plt.plot(train_sizes_abs, train_scores.mean(axis=1), label='Train')
plt.plot(train_sizes_abs, test_scores.mean(axis=1), label='Test')
plt.legend()
plt.xlabel('Размер обучающей выборки')
plt.ylabel('ROC AUC')
plt.title('Learning curves');

2\. Часто несбалансированные по классам выборки приводят к различным проблемам при обучении моделей. Давайте попробуем по-разному обработать выборку, поиграть с распределением объектов по классам и сделать выводы о том, как соотношение классов влияет на качество модели.

2.1\. Задайте веса объектам так, чтобы соотношение классов с учетом весов объектов изменилось. Попробуйте не менее трёх различных вариантов весов. Меняются ли результаты классификации? Как это сказывается на качестве модели? Какой вариант выглядит наиболее оптимальным с точки зрения качества?

In [ ]:
y_train.value_counts()[1] / y_train.value_counts()[-1], y_train.value_counts()[-1] / y_train.value_counts()[1]

In [ ]:
catboost_classifier = CatBoostClassifier(
    max_depth=6, 
    n_estimators=100, 
    cat_features=df_train.select_dtypes(include='object').columns.to_list(),
    logging_level='Silent',
#     auto_class_weights='SqrtBalanced',
    scale_pos_weight=0.08
)

In [ ]:
%%time
catboost_classifier.fit(X_train, y_train)
print(roc_auc_score(y_valid, catboost_classifier.predict_proba(X_valid)[:, 1]))

In [ ]:
%%time
scores = cross_val_score(estimator=catboost_classifier, X=X_train, y=y_train, cv=cv, n_jobs=-1, 
                         scoring='roc_auc')
print(scores, '\n', scores.mean())

- auto_class_weights: SqrtBalanced, Balanced
- scale_pos_weight=12

Нет изменений или ухудшение

- scale_pos_weight=0.08

Улучшение

2.2\. Примените к выборке технологию undersampling: для этого нужно убрать из обучения некоторое количество объектов большего класса таким образом, чтобы соотношение классов изменилось. Попробуйте не менее трёх различных вариантов undersampling (варианты могут отличаться как по количество отфильтрованных объектов, так и по принципу выборка объектов для отсеивания из выборки). Меняются ли результаты классификации? Как это сказывается на качестве модели? Какой вариант выглядит наиболее оптимальным с точки зрения качества?

In [ ]:
THRESH = 73

df_n_churn = df[y == -1]
df_undersampling = pd.concat([df_n_churn.dropna(thresh=THRESH), df[y == 1]])
df_train, df_ho, y_train, y_valid = train_test_split(df_undersampling, y[df_undersampling.index], test_size=5000)
df_undersampling

In [ ]:
%%time
X_train_all = preprocessor_pipeline.fit_transform(df_undersampling)
X_train_all = pd.DataFrame(X_train_all, columns=preprocessor_pipeline['nan_columns_dropper'].cols)#.astype('int')
X_train = preprocessor_pipeline.fit_transform(df_train)
X_train = pd.DataFrame(X_train, columns=preprocessor_pipeline['nan_columns_dropper'].cols)#.astype('int')
X_valid = preprocessor_pipeline.fit_transform(df_ho)
X_valid = pd.DataFrame(X_valid, columns=preprocessor_pipeline['nan_columns_dropper'].cols)#.astype('int')
X_train

In [ ]:
catboost_classifier = CatBoostClassifier(
    max_depth=6, 
    n_estimators=100, 
    cat_features=df_train.select_dtypes(include='object').columns.to_list(),
    logging_level='Silent',
)

In [ ]:
%%time
catboost_classifier.fit(X_train, y_train)
print(roc_auc_score(y_valid, catboost_classifier.predict_proba(X_valid)[:, 1]))

In [ ]:
%%time
scores = cross_val_score(estimator=catboost_classifier, X=X_train, y=y_train, cv=cv, n_jobs=-1, 
                         scoring='roc_auc')
print(scores, '\n', scores.mean())

|THRESH|ROC AUC|
|-|-|
|70|0.8|
|73|0.9|

3\. Теперь перейдем к работе с признаками. Ранее вы реализовали несколько стратегий для обработки пропущенных значений. Сравните эти стратегии между собой с помощью оценки качества моделей кросс-валидации, построенных на датасетах с использованием различных стратегий. Как обработка пропущенных значений сказывается на качестве модели? Какой вариант выглядит наиболее оптимальным с точки зрения качества?

4\. Также вы уже реализовали несколько стратегий для обработки категориальных признаков. Сравните эти стратегии между собой с помощью оценки качества моделей по кросс-валидации, построенных на датасетах с использованием различных стратегий. Как обработка категориальных признаков сказывается на качестве модели? Какой вариант выглядит наиболее оптимальным с точки зрения качества?

5\. Все ли признаки оказались полезными для построения моделей? Проведите процедуру отбора признаков, попробуйте разные варианты отбора (обратите внимание на модуль `sklearn.feature_selection`). Например, можно выбрасывать случайные признаки или строить отбор на основе l1-регуляризации - отфильтровать из обучения признаки, которые получат нулевой вес при построении регрессии с l1-регуляризацией (`sklearn.linear_model.Lasso`). И всегда можно придумать что-то своё=) Попробуйте как минимум 2 различные стратегии, сравните результаты. Помог ли отбор признаков улучшить качество модели? Поясните свой ответ.

6\. Подберите оптимальные параметры модели. Обратите внимание, что в зависимости от того, как вы обработали исходные данные, сделали ли балансировку классов, сколько объектов оставили в обучающей выборке и др. оптимальные значения параметров могут меняться. Возьмите наилучшее из ваших решений на текущий момент и проведите процедуру подбора параметров модели (обратите внимание на `sklearn.model_selection.GridSearchCV`) Как подбор параметров повлиял на качество модели?

7\. Предложите методику оценки того, какие признаки внесли наибольший вклад в модель (например, это могут быть веса в случае регрессии, а также большое количество моделей реализуют метод `feature_importances_` - оценка важности признаков). На основе предложенной методики проанализируйте, какие признаки внесли больший вклад в модель, а какие меньший?

8\. Напоследок давайте посмотрим на объекты. На каких объектах достигается наибольшая ошибка классификации? Есть ли межу этими объектами что-то общее? Видны ли какие-либо закономерности? Предположите, почему наибольшая ошибка достигается именно на этих объектах. В данном случае "наибольшую" ошибку можно понимать как отнесение объекта с чужому классу с большой долей уверенности (с высокой вероятностью).

9\. По итогам проведенных экспериментов постройте финальную решение - модель с наилучшим качеством. Укажите, какие преобразования данных, параметры и пр. вы выбрали для построения финальной модели.

10\. Подумайте, можно ли еще улучшить модель? Что для этого можно сделать? 

# Тест

In [ ]:
def write_to_submission_file(predicted_labels, out_file,
                             target='result', index_label="id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(0, predicted_labels.shape[0]),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
df_test = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_test_data.csv'), index_col=0)
df_test

In [ ]:
%%time
X_test = preprocessor_pipeline.fit_transform(df_test)
X_test = pd.DataFrame(X_test, columns=preprocessor_pipeline['nan_columns_dropper'].cols)#.astype('int')
X_test

In [ ]:
%%time
# catboost_classifier.fit(X_train_all, y)
prediction = catboost_classifier.predict_proba(X_test)

In [ ]:
write_to_submission_file(prediction[:, 1], 'result.csv')